## Estimate the amount of oil cargo carreid by ATBs by comparing DOE cargo transfers to tank capacities

This notebook is a compliment to [ATB_FullEmptyRatio_AISplot.ipynb](https://github.com/MIDOSS/analysis-rachael/blob/main/notebooks/monte_carlo/ATB_FullEmptyRatio_AISplot.ipynb) and [ATB_FullEmptyRatio.ipynb](https://github.com/MIDOSS/analysis-rachael/blob/main/notebooks/monte_carlo/ATB_FullEmptyRatio.ipynb). See also [Susan's spreadsheet](https://docs.google.com/spreadsheets/d/1nyp_gtYGsc4xvN7eLj48I6eWmitgb-Plut-vmX79UMM/edit#gid=0) on oil transfer 

Use `analysis-rachael/env/monte_carlo.yaml` to create an environment for this notebook:
```
 conda env create -f [analysis-rachael/env/monte_carlo.yaml]
 ```
or, to activate this environment, use
```
 conda activate monte_carlo
 ```
To deactivate an active environment, use
```
 conda deactivate
 ```

In [1]:
import pandas 
import numpy 
import matplotlib.pyplot as plt
import yaml
from pathlib import Path
import datetime

# import functions for querying DOE and monte-carlo dataframes
from monte_carlo_utils import get_DOE_df, get_DOE_atb

In [2]:
# Dept. of Ecology data files
DOE_dir  = Path('/Users/rmueller/Data/MIDOSS/DeptOfEcology/')
DOE_2018_xlsx = DOE_dir/'MuellerTrans4-30-20.xlsx'

# ATB spreadsheet
atb_capacity_xlsx = Path(
    '/Users/rmueller/Data/MIDOSS/marine_transport_data/ATB_oil_capacity.xlsx'
)

# Facility names and lat/lon information file
facilities_xlsx = Path(
    '/Users/rmueller/Data/MIDOSS/marine_transport_data/'
    'Oil_Transfer_Facilities.xlsx'
)

# Constants
bbl2gal = 42  #barrel to gallons

### Create histogram of percent fill for each transfer, by each vessel, for import and export transfers combined
- Create dictionary `capacity` with tank barge and oil capacity as key:value pairs
- Load DOE data and create dataframes for each tank barge
- Group dataframes by AntID
- Estimate fullness per transfer based on cargo capacity information 
    - NOTE: Assumption here is that ATBs fill at one location rather than, e.g., two separate locations
    - (save transfer dates and tank barge names in each bin? )
- Create a histogram of percent full for imports, exports and combined for each vessel
- Add up the combined histograms across all vessels

#### Create dictionary `capacity` with tank barge and oil capacity as key:value pairs

In [3]:
atb_df = pandas.read_excel(
    atb_capacity_xlsx,
    sheet_name='Cargo Capacities', 
    usecols="A,H"
    )
capacity = {}
for idx,name in enumerate(atb_df['DOE name']):
    capacity[name] = atb_df['95% or 96% Capacity (bbls)']

#### Load DOE data and create import/export dataframes for each tank barge

In [4]:
# load data
[imports, exports]=get_DOE_atb(
    DOE_2018_xlsx, 
    facilities_xlsx, 
    transfer_type = 'cargo', 
    facilities='selected'
)

In [6]:
# create dictionary of cargo fullness by tank barge 
imports_fill = {}
exports_fill = {}
fullness = {}
# extract the important stuff
imports = imports[['AntID','Deliverer','Receiver','Product','TransferQtyInGallon']]
exports = exports[['AntID','Deliverer','Receiver','Product','TransferQtyInGallon']]
# define way to group dataframe
aggregation_functions = {
    'AntID': 'first', 
    'Deliverer': 'first', 
    'Receiver': 'first',
    'Product': lambda x: x.tolist(),
    'TransferQtyInGallon': 'sum'
}
# first group by transfer ID (AntID) and add `TransferQtyInGallon` for each AntID
imports_byAntID = imports.groupby(imports['AntID']).aggregate(aggregation_functions)
exports_byAntID = exports.groupby(exports['AntID']).aggregate(aggregation_functions)
# loop through vessel names and create dictionary list of %full values for each AntID
for idx,name in enumerate(atb_df['DOE name']):
    # imports
    imports_fill[name] = imports_byAntID.loc[
        imports_byAntID['Deliverer']==name,
        ['TransferQtyInGallon','Receiver','Product']
    ].rename(columns={'Receiver':'Facility'})
    imports_fill[name]['PercentFilled'] = numpy.zeros(len(imports_fill[name]['TransferQtyInGallon']))  
    imports_fill[name]['PercentFilled']=numpy.asarray(
        [x / capacity[name].values for x in imports_fill[name]['TransferQtyInGallon']]
    )
    imports_fill[name]['direction'] = 'import'
    # exports
    exports_fill[name] = exports_byAntID.loc[
        exports_byAntID['Receiver']==name,
        ['TransferQtyInGallon','Deliverer','Product']
    ].rename(columns={'Deliverer':'Facility'})  
    exports_fill[name]['PercentFilled'] = numpy.zeros(len(exports_fill[name]['TransferQtyInGallon']))
    exports_fill[name]['PercentFilled'] = numpy.asarray([
        x / capacity[name].values for x in exports_fill[name]['TransferQtyInGallon']
    ])
    exports_fill[name]['direction'] = 'export'
    # combine imports and exports
    frames = [imports_fill[name], exports_fill[name]]
    fullness[name] = pandas.concat(frames).reset_index()

In [7]:
fullness['ATB BARGE KIRBY 185-01']

,AntID,TransferQtyInGallon,Facility,Product,PercentFilled,direction
0,177755,4200000.0,Shell Puget Sound Refinery,[CRUDE OIL],52.203744,import
1,180010,4200000.0,Shell Puget Sound Refinery,[Cat Feed/VGO],52.203744,import
2,180818,6812610.0,Shell Puget Sound Refinery,[CRUDE OIL],84.677083,import
3,183818,1470000.0,Kinder Morgan Liquids Terminal - Harbor Island,[BUNKER OIL/HFO],18.271310,import
4,184777,3454500.0,Shell Puget Sound Refinery,"[Cat Feed/VGO, DIESEL/MARINE GAS OIL]",42.937579,import
5,173493,6510000.0,Shell Puget Sound Refinery,"[BUNKER OIL/HFO, DIESEL/MARINE GAS OIL]",80.915803,export
6,174751,5376000.0,Shell Puget Sound Refinery,"[BUNKER OIL/HFO, DIESEL/MARINE GAS OIL]",66.820792,export
7,175367,630000.0,Shell Puget Sound Refinery,[BUNKER OIL/HFO],7.830562,export
8,175873,7350000.0,Shell Puget Sound Refinery,"[CRUDE OIL, DIESEL/MARINE GAS OIL, BUNKER OIL/...",91.356552,export
9,179928,4200000.0,Shell Puget Sound Refinery,[DIESEL/MARINE GAS OIL],52.203744,export
